# Feature Store - Feature Retrieval Examples

This notebook demonstrates retrieving features from Vertex AI Feature Store online serving (BigTable).

## Prerequisites

Before running this notebook, ensure you have executed:
1. `fs-data-types.py` - Creates feature views for different BigQuery source types (internal table, external table, logical view)
2. `fs-data-shapes.py` - Creates feature views for different data shapes (all features, odd features, even features)

Both scripts set up the BigTable online store and sync feature data for online serving.

## Retrieval Patterns

This notebook covers two common online feature retrieval patterns:
1. **Single Entity Retrieval** - Synchronous lookup for one entity using the SDK
2. **Multi-Entity Streaming Retrieval** - Efficient batch retrieval for multiple entities using the streaming API

---
## Imports

In [1]:
import subprocess
from google.cloud import aiplatform
from vertexai.resources.preview import feature_store
from google.cloud.aiplatform_v1beta1 import FeatureOnlineStoreServiceClient
from google.cloud.aiplatform_v1beta1.types import feature_online_store_service as feature_online_store_service_pb2

---
## Setup

Initialize project configuration, clients, and validate that the feature store and all feature views are ready.

In [2]:
# Configuration
LOCATION = 'us-central1'
PROJECT_ID = subprocess.run(['gcloud', 'config', 'get-value', 'project'], capture_output=True, text=True).stdout.strip()

# Initialize clients
aiplatform.init(project=PROJECT_ID, location=LOCATION)

print(f"Project: {PROJECT_ID}")
print(f"Location: {LOCATION}")

Project: statmike-mlops-349915
Location: us-central1


In [3]:
# Get feature store
FEATURE_STORE_NAME = PROJECT_ID.replace('-', '_') + '_bigtable'

try:
    online_store = feature_store.FeatureOnlineStore(name=FEATURE_STORE_NAME)
    if online_store.feature_online_store_type.name == 'BIGTABLE':
        print(f"✓ Found BigTable feature store: {FEATURE_STORE_NAME}")
        print(f"  Resource: {online_store.resource_name}")
    else:
        raise Exception(f"Store '{FEATURE_STORE_NAME}' is {online_store.feature_online_store_type.name}, not BIGTABLE")
except Exception as e:
    print(f"✗ Error: Could not find feature store '{FEATURE_STORE_NAME}'")
    print(f"  Please run fs-data-types.py and fs-data-shapes.py first")
    raise e

✓ Found BigTable feature store: statmike_mlops_349915_bigtable
  Resource: projects/1026793852137/locations/us-central1/featureOnlineStores/statmike_mlops_349915_bigtable


In [4]:
# Validate all feature views exist
FEATURE_VIEW_NAMES = [
    'all_features',
    'odd_features', 
    'even_features',
    'example_internal_table',
    'example_external_table',
    'example_logical_view'
]

feature_views = {}
print("Validating feature views...")

for view_name in FEATURE_VIEW_NAMES:
    try:
        view = feature_store.FeatureView(
            name=view_name,
            feature_online_store_id=online_store.resource_name
        )
        feature_views[view_name] = view
        print(f"  ✓ {view_name}")
    except Exception as e:
        print(f"  ✗ {view_name} - Not found")
        print(f"    Error: {e}")
        raise Exception(f"Feature view '{view_name}' not found. Please run setup scripts first.")

print(f"\n✓ All {len(feature_views)} feature views are ready for serving")

Validating feature views...
  ✓ all_features
  ✓ odd_features
  ✓ even_features
  ✓ example_internal_table
  ✓ example_external_table
  ✓ example_logical_view

✓ All 6 feature views are ready for serving


In [5]:
# Initialize streaming client for multi-entity retrieval
data_client = FeatureOnlineStoreServiceClient(
    client_options={"api_endpoint": f"{LOCATION}-aiplatform.googleapis.com"}
)
print("✓ Streaming client initialized")

✓ Streaming client initialized


### Helper Functions

In [21]:
def parse_feature_value(feature_struct: dict):
    """Extract the typed value from a feature's value dictionary."""
    value_dict = feature_struct.get('value', {})
    if not value_dict:
        return None
    
    # Handle int64 conversion
    if 'int64_value' in value_dict:
        return int(value_dict['int64_value'])
    
    # Return first non-None value
    try:
        return next(iter(value_dict.values()))
    except StopIteration:
        return None


def display_features(entity_id: str, features: dict):
    """Display features for a single entity in a clean format."""
    print(f"\nEntity: {entity_id}")
    print(f"Features retrieved: {len(features)}")

    if features:
        print(f"Features: {features}")
    else:
        print("No features found")


def display_single_entity_result(view_name: str, key: list, feature_view: feature_store.FeatureView):
    """Retrieve and display features for a single entity."""
    print(f"Retrieving from '{view_name}' for entity: {key[0]}")
    
    try:
        # Read features from view
        raw_features = feature_view.read(key=key).to_dict().get('features', [])
        
        # Parse into dictionary (exclude feature_timestamp)
        features = {
            f.get('name'): parse_feature_value(f)
            for f in raw_features
            if f.get('name') and f.get('name') != 'feature_timestamp'
        }
        
        display_features(key[0], features)
        
    except Exception as e:
        print(f"✗ Error retrieving features: {e}")


def display_multi_entity_results(view_name: str, entity_keys: list, feature_view: feature_store.FeatureView):
    """Retrieve and display features for multiple entities using streaming API."""
    print(f"Retrieving from '{view_name}' for entities: {[k[0] for k in entity_keys]}")
    print("-" * 60)
    
    try:
        # Prepare streaming requests
        requests = [
            feature_online_store_service_pb2.StreamingFetchFeatureValuesRequest(
                feature_view=feature_view.resource_name,
                data_keys=[feature_online_store_service_pb2.FeatureViewDataKey(key=k) for k in key_list]
            )
            for key_list in entity_keys
        ]
        
        # Retrieve streaming responses
        responses = data_client.streaming_fetch_feature_values(requests=iter(requests))
        
        # Process each response
        for i, response in enumerate(responses):
            # Handle errors
            if response.status and response.status.code != 0:
                print(f"\n✗ Request for entity {entity_keys[i]} failed: {response.status.message}")
                continue
            
            # Process each entity in the response
            for data_item in response.data:
                entity_id = data_item.data_key.key if data_item.data_key else ['unknown']
                
                # Parse features
                features = {
                    feature.name: (
                        feature.value.string_value or
                        feature.value.int64_value or
                        feature.value.double_value or
                        feature.value.bool_value or
                        None
                    )
                    for feature in data_item.key_values.features
                    if feature.name != 'feature_timestamp'
                } if data_item.key_values else {}
                
                display_features(entity_id[0] if isinstance(entity_id, list) else entity_id, features)
        
        print("-" * 60)
        
    except Exception as e:
        print(f"✗ Error during streaming retrieval: {e}")

print("✓ Helper functions loaded")

✓ Helper functions loaded


---
## Single Entity Retrieval

Retrieve features for a single entity using the synchronous `feature_view.read()` method. This is ideal for:
- Real-time predictions for individual requests
- Low-latency serving scenarios
- Simple lookup operations

We'll retrieve features for `entity-1` from all 6 feature views.

In [22]:
# Single entity to retrieve
SINGLE_ENTITY_KEY = ['entity-1']

### All Features View
Contains all 30 features from fs-data-shapes.py (features 1-30 across all data shapes)

In [23]:
display_single_entity_result('all_features', SINGLE_ENTITY_KEY, feature_views['all_features'])

Retrieving from 'all_features' for entity: entity-1

Entity: entity-1
Features retrieved: 30
Features: {'feature_1': True, 'feature_4': 0.85, 'feature_3': 'High', 'feature_2': 75, 'feature_5': False, 'feature_10': False, 'feature_7': 5, 'feature_9': 0.75, 'feature_8': 'Medium', 'feature_6': True, 'feature_22': 190, 'feature_21': True, 'feature_23': 'Gamma', 'feature_24': 9.8, 'feature_25': True, 'feature_26': True, 'feature_27': 123, 'feature_28': 'Red', 'feature_29': 0.65, 'feature_30': False, 'feature_11': True, 'feature_13': 'Low', 'feature_12': 78, 'feature_15': False, 'feature_14': 0.87, 'feature_17': 19, 'feature_18': 'Excellent', 'feature_19': 55.5, 'feature_20': False, 'feature_16': True}


### Odd Features View
Contains only odd-numbered features (1, 3, 5, 7, ..., 29)

In [24]:
display_single_entity_result('odd_features', SINGLE_ENTITY_KEY, feature_views['odd_features'])

Retrieving from 'odd_features' for entity: entity-1

Entity: entity-1
Features retrieved: 15
Features: {'feature_1': True, 'feature_3': 'High', 'feature_5': False, 'feature_7': 5, 'feature_9': 0.75, 'feature_21': True, 'feature_23': 'Gamma', 'feature_25': True, 'feature_27': 123, 'feature_29': 0.65, 'feature_11': True, 'feature_13': 'Low', 'feature_15': False, 'feature_17': 19, 'feature_19': 55.5}


### Even Features View
Contains only even-numbered features (2, 4, 6, 8, ..., 30)

In [25]:
display_single_entity_result('even_features', SINGLE_ENTITY_KEY, feature_views['even_features'])

Retrieving from 'even_features' for entity: entity-1

Entity: entity-1
Features retrieved: 15
Features: {'feature_2': 75, 'feature_4': 0.85, 'feature_10': False, 'feature_6': True, 'feature_8': 'Medium', 'feature_22': 190, 'feature_24': 9.8, 'feature_26': True, 'feature_28': 'Red', 'feature_30': False, 'feature_12': 78, 'feature_14': 0.87, 'feature_16': True, 'feature_18': 'Excellent', 'feature_20': False}


### Internal Table View
Features sourced from a standard BigQuery internal table (features 1-4)

In [26]:
display_single_entity_result('example_internal_table', SINGLE_ENTITY_KEY, feature_views['example_internal_table'])

Retrieving from 'example_internal_table' for entity: entity-1

Entity: entity-1
Features retrieved: 4
Features: {'feature_1': 42, 'feature_2': 0.5, 'feature_3': 'a', 'feature_4': True}


### External Table View
Features sourced from a BigQuery external table pointing to GCS (features 1-4)

In [27]:
display_single_entity_result('example_external_table', SINGLE_ENTITY_KEY, feature_views['example_external_table'])

Retrieving from 'example_external_table' for entity: entity-1

Entity: entity-1
Features retrieved: 4
Features: {'feature_1': 42, 'feature_2': 0.5, 'feature_3': 'a', 'feature_4': True}


### Logical View
Features sourced from a BigQuery logical view with transformations (features 1-4)

In [28]:
display_single_entity_result('example_logical_view', SINGLE_ENTITY_KEY, feature_views['example_logical_view'])

Retrieving from 'example_logical_view' for entity: entity-1

Entity: entity-1
Features retrieved: 4
Features: {'feature_1': 42, 'feature_2': 0.5, 'feature_3': 'a', 'feature_4': True}


---
## Multi-Entity Streaming Retrieval

Retrieve features for multiple entities in a single batch using the streaming API. This is ideal for:
- Batch prediction workloads
- Efficient retrieval of features for many entities
- Higher throughput scenarios

We'll retrieve features for 3 entities (`entity-1`, `entity-2`, `entity-3`) from all 6 feature views.

In [29]:
# Multiple entities to retrieve
MULTI_ENTITY_KEYS = [['entity-1'], ['entity-2'], ['entity-3']]

### All Features View
Streaming retrieval of all 30 features for 3 entities

In [30]:
display_multi_entity_results('all_features', MULTI_ENTITY_KEYS, feature_views['all_features'])

Retrieving from 'all_features' for entities: ['entity-1', 'entity-2', 'entity-3']
------------------------------------------------------------

Entity: entity-1
Features retrieved: 30
Features: {'feature_1': True, 'feature_4': 0.85, 'feature_3': 'High', 'feature_2': 75, 'feature_5': None, 'feature_10': None, 'feature_7': 5, 'feature_9': 0.75, 'feature_8': 'Medium', 'feature_6': True, 'feature_22': 190, 'feature_21': True, 'feature_23': 'Gamma', 'feature_24': 9.8, 'feature_25': True, 'feature_26': True, 'feature_27': 123, 'feature_28': 'Red', 'feature_29': 0.65, 'feature_30': None, 'feature_11': True, 'feature_13': 'Low', 'feature_12': 78, 'feature_15': None, 'feature_14': 0.87, 'feature_17': 19, 'feature_18': 'Excellent', 'feature_19': 55.5, 'feature_20': None, 'feature_16': True}

Entity: entity-2
Features retrieved: 30
Features: {'feature_1': None, 'feature_4': 0.15, 'feature_3': 'Low', 'feature_2': 20, 'feature_5': True, 'feature_10': True, 'feature_7': 2, 'feature_9': 0.15, 'featur

### Odd Features View
Streaming retrieval of odd-numbered features for 3 entities

In [31]:
display_multi_entity_results('odd_features', MULTI_ENTITY_KEYS, feature_views['odd_features'])

Retrieving from 'odd_features' for entities: ['entity-1', 'entity-2', 'entity-3']
------------------------------------------------------------

Entity: entity-2
Features retrieved: 15
Features: {'feature_1': None, 'feature_3': 'Low', 'feature_5': True, 'feature_7': 2, 'feature_9': 0.15, 'feature_21': None, 'feature_23': 'Delta', 'feature_25': True, 'feature_27': 200, 'feature_29': -0.5, 'feature_11': None, 'feature_13': 'Low', 'feature_15': True, 'feature_17': 42, 'feature_19': 70.3}

Entity: entity-1
Features retrieved: 15
Features: {'feature_1': True, 'feature_3': 'High', 'feature_5': None, 'feature_7': 5, 'feature_9': 0.75, 'feature_21': True, 'feature_23': 'Gamma', 'feature_25': True, 'feature_27': 123, 'feature_29': 0.65, 'feature_11': True, 'feature_13': 'Low', 'feature_15': None, 'feature_17': 19, 'feature_19': 55.5}

Entity: entity-3
Features retrieved: 15
Features: {'feature_1': True, 'feature_3': 'High', 'feature_5': True, 'feature_7': 9, 'feature_9': 0.95, 'feature_21': True

### Even Features View
Streaming retrieval of even-numbered features for 3 entities

In [32]:
display_multi_entity_results('even_features', MULTI_ENTITY_KEYS, feature_views['even_features'])

Retrieving from 'even_features' for entities: ['entity-1', 'entity-2', 'entity-3']
------------------------------------------------------------

Entity: entity-1
Features retrieved: 15
Features: {'feature_2': 75, 'feature_4': 0.85, 'feature_10': None, 'feature_6': True, 'feature_8': 'Medium', 'feature_22': 190, 'feature_24': 9.8, 'feature_26': True, 'feature_28': 'Red', 'feature_30': None, 'feature_12': 78, 'feature_14': 0.87, 'feature_16': True, 'feature_18': 'Excellent', 'feature_20': None}

Entity: entity-3
Features retrieved: 15
Features: {'feature_2': 95, 'feature_4': 0.99, 'feature_10': True, 'feature_6': True, 'feature_8': 'High', 'feature_22': 135, 'feature_24': 9.1, 'feature_26': True, 'feature_28': 'Green', 'feature_30': None, 'feature_12': 5, 'feature_14': 0.09, 'feature_16': True, 'feature_18': 'Excellent', 'feature_20': None}

Entity: entity-2
Features retrieved: 15
Features: {'feature_2': 20, 'feature_4': 0.15, 'feature_10': True, 'feature_6': None, 'feature_8': 'Low', 'f

### Internal Table View
Streaming retrieval from internal table source for 3 entities

In [33]:
display_multi_entity_results('example_internal_table', MULTI_ENTITY_KEYS, feature_views['example_internal_table'])

Retrieving from 'example_internal_table' for entities: ['entity-1', 'entity-2', 'entity-3']
------------------------------------------------------------

Entity: entity-1
Features retrieved: 4
Features: {'feature_1': 42, 'feature_2': 0.5, 'feature_3': 'a', 'feature_4': True}

Entity: entity-2
Features retrieved: 4
Features: {'feature_1': 15, 'feature_2': 0.9, 'feature_3': 'b', 'feature_4': None}

Entity: entity-3
Features retrieved: 4
Features: {'feature_1': 88, 'feature_2': 0.1, 'feature_3': 'c', 'feature_4': True}
------------------------------------------------------------


### External Table View
Streaming retrieval from external table source for 3 entities

In [34]:
display_multi_entity_results('example_external_table', MULTI_ENTITY_KEYS, feature_views['example_external_table'])

Retrieving from 'example_external_table' for entities: ['entity-1', 'entity-2', 'entity-3']
------------------------------------------------------------

Entity: entity-2
Features retrieved: 4
Features: {'feature_1': 15, 'feature_2': 0.9, 'feature_3': 'b', 'feature_4': None}

Entity: entity-1
Features retrieved: 4
Features: {'feature_1': 42, 'feature_2': 0.5, 'feature_3': 'a', 'feature_4': True}

Entity: entity-3
Features retrieved: 4
Features: {'feature_1': 88, 'feature_2': 0.1, 'feature_3': 'c', 'feature_4': True}
------------------------------------------------------------


### Logical View
Streaming retrieval from logical view source for 3 entities

In [35]:
display_multi_entity_results('example_logical_view', MULTI_ENTITY_KEYS, feature_views['example_logical_view'])

Retrieving from 'example_logical_view' for entities: ['entity-1', 'entity-2', 'entity-3']
------------------------------------------------------------

Entity: entity-1
Features retrieved: 4
Features: {'feature_1': 42, 'feature_2': 0.5, 'feature_3': 'a', 'feature_4': True}

Entity: entity-2
Features retrieved: 4
Features: {'feature_1': 15, 'feature_2': 0.9, 'feature_3': 'b', 'feature_4': None}

Entity: entity-3
Features retrieved: 4
Features: {'feature_1': 88, 'feature_2': 0.1, 'feature_3': 'c', 'feature_4': True}
------------------------------------------------------------


---
## Summary

This notebook demonstrated two core patterns for online feature retrieval from Vertex AI Feature Store:

1. **Single Entity Retrieval** - Simple, synchronous SDK method suitable for real-time individual predictions
2. **Multi-Entity Streaming** - Efficient batch retrieval using the streaming API for higher throughput

Both patterns work consistently across different feature view configurations:
- Different feature subsets (all, odd, even)
- Different BigQuery source types (internal tables, external tables, logical views)
- Different data shapes (dense, sparse, EAV)

### Key Takeaways

- Feature views abstract away the underlying data source complexity
- The same retrieval APIs work regardless of source table type or data shape
- Single entity retrieval provides simplicity for low-latency use cases
- Streaming retrieval provides efficiency for batch prediction workloads
- All features are served from BigTable for consistent low-latency access